# 해야할 일들
1. 희수님께 테스트셋 받으면 정확도 나오게 하기 (필수)
2. 엑셀로 결과 저장시, 파라미터 세팅을 포함시키거나, 사용된 파이썬파일을 포함시켜서 매번 새로운 폴더에 넣도록 해보기 (고려)
3. 태그별 레이블을 지정해서 전처리 함수에 인자로 받도록 수정해보기 (고려)

# Import library & Load data

In [1]:
#@title
!pip install transformers
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import transformers
from transformers import AutoTokenizer, AdamW, RobertaForSequenceClassification

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

from tqdm.notebook import tqdm

import os
import re

import random

from google.colab import drive

drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 전처리용 함수 정의 (안봐도 됨)
# 희수님이 레이블링한엑셀파일, 문장의미태깅데이터셋 => 이 모델에 맞는 형식의 csv 4개 생성

In [2]:
#@title
#zipfile : 태깅데이터셋zip 경로 // excelfile : 희수님이 보내주신 엑셀파일 경로 // sep : 출력파일에 사용될 구분자 // add_from_excel : 엑셀파일을 사용할지 무시할지 여부 // 테스트셋에 사용할 비율
#나머지는 출력파일 경로랑 레이블 개수 등
def get_csv(zipfile,excelfile,sep,add_from_excel,usage_ratio,train_out,topic_dict_out,num_labels):
  dir_path = '/content/data'  # 태깅 데이터셋 압축해제될 폴더
  if not os.path.exists(dir_path): os.makedirs(dir_path)
  !unzip -oqq {zipfile} -d {dir_path}

  #출력파일경로를 자동생성
  if not os.path.exists(os.path.dirname(train_out)): os.makedirs(os.path.dirname(train_out))
  if not os.path.exists(os.path.dirname(test_out)): os.makedirs(os.path.dirname(test_out))
  if not os.path.exists(os.path.dirname(sample_submission_out)): os.makedirs(os.path.dirname(sample_submission_out))
  if not os.path.exists(os.path.dirname(topic_dict_out)): os.makedirs(os.path.dirname(topic_dict_out))

  dataset=[]
  if add_from_excel is True:
    df = pd.read_excel(excelfile)
    df = df.iloc[:df.shape[0]] #첫줄은 인덱스로받아들이고 나머지줄은 데이터로받아들임
    label0 = df[df['후속연구'] == 'N']['문장'].tolist()
    for i in range(len(label0)):
      label0[i]=label0[i]+sep+'0\n'
    label1 = df[df['후속연구'] == 'Y']['문장'].tolist()
    for i in range(len(label1)):
      label1[i]=label1[i]+sep+'1\n'
    dataset=label0[:]+label1[:]
  print('엑셀에서 추가된 데이터 개수:', len(dataset))

  tags=[] #태그 종류 확인용
  for root_dir, sub_dirs, filenames in os.walk(dir_path):
      for filename in filenames:
          if filename[-4:]!='json': continue #json파일만 걸러내기용
          filepath = os.path.join(root_dir, filename)
          input_file_name = filepath
          with open(input_file_name) as file_in:
            for line in file_in.readlines():
              if '"sentence"' in line:
                text=line[14:-3] # ,\n 으로 끝나니까 -3까지슬라이싱
              elif '"tag"' in line:
                if line not in tags: tags.append(line) #태그 종류 확인용
                if '문제 정의' in line or '가설 설정' in line or '기술 정의' in line:
                  topic_idx=0
                elif '후속연구' in line or '성능/효과' in line:
                  topic_idx=1
                else: topic_idx=2 #연구방법에 관한 것들을 학습데이터에서 제외시키기 위함
                # if '후속연구' in line: #위처럼 했더니 결과가 잘 안나왔음
                #   topic_idx=1
                # else:
                #   topic_idx=0
              # elif line=='},{\n' or line=='}': #연구방법에 관한 것들을 학습데이터에서 제외시키려면 주석처리, 포함시키려면 주석해제
              elif (line=='},{\n' or line=='}') and topic_idx!=2: #연구방법에 관한 것들을 학습데이터에서 제외시키려면 주석해제, 포함시키려면 주석처리
                dataset.append(text+sep+str(topic_idx)+'\n')

  usecut=int(usage_ratio*len(dataset))
  dataset=dataset[:usecut] #엑셀에서 추가한 것들은 usage_ratio에 영향 안받도록 함
  random.seed(42)
  random.shuffle(dataset)

  index=0
  with open(train_out,'w') as f:
    f.write('index'+sep+'text'+sep+'topic_idx\n')
    for i in dataset:
      f.write(str(index)+sep+i)
      index+=1

  with open(topic_dict_out,'w') as f:
    # f.write('topic',+sep+'topic_idx\n제안안함'+sep+'0\n제안함'+sep+'1\n그외나머지'+sep+'2\n')
    f.write('topic'+sep+'topic_idx\n제안안함'+sep+'0\n제안함'+sep+'1\n') #연구방법에 해당하는것들 제외시키려면 이거로 수행.
    for i in range(num_labels):
      if i>1:
        f.write('label'+str(i)+sep+str(i)+'\n')


# 전처리용 함수 정의 (안봐도 됨)
# 희수님 크롤링 데이터 => 10문장 추출해서 테스트셋용 csv 파일로 저장 (구분자 재지정)

In [3]:
#@title
#각각의 기사 당, 마침표 기준으로 마지막 10문장 뽑아서 저장.
#가능하다면 마지막 부분에 있는 필요없는 부분(이메일, 기자 이름, 기자 한마디 등) 제거하여 저장하기
  #기본 가정 : 크롤링 된 기사 텍스트의 중반 이후에서 처음 등장하는 이메일 주소 직전 문장까지가 기사의 내용이라고 가정
  #한계 : 기사 자체가 이메일 주소에 관한 내용을 다룬다면 제대로 내용을 추출할 수 없음.


#ㅡㅡㅡㅡㅡㅡㅡ이메일 감지ㅡㅡㅡㅡㅡㅡㅡ
def find_emails(string):
    email_pattern = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}'
    return re.findall(email_pattern, string) #이메일 미포함이면 []반환
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

#ㅡㅡㅡㅡ마침표가 새로운 문장을 의미하는 것인지를 판단
def is_new_sentence(dot_idx,content): #content는 문자열이 들어감.
  #ㅡㅡㅡㅡㅡㅡㅡ이메일에 사용된 마침표인지 판단
  start=dot_idx-1 
  end=dot_idx+1
  while start>=0: 
    if content[start]==' ':
      break
    else: start-=1
  while end<len(content):
    if content[end]==' ':
      break
    else: end+=1
  start+=1
  if find_emails(content[start:end])!=[]:
    return False
  #ㅡㅡㅡㅡㅡㅡ날짜나 퍼센트 소수점 등 몇가지 문제가 있어서 그냥 통상적으로 몇가지 화이트리스트만 갖고 해야겠음
  white_list_1str=['다', '나', '까', '"', "'", '?', '도', '요', '죠', '라', '”', '“', '‘', '’']
  white_list_2str=[i+' ' for i in white_list_1str]
  if dot_idx>=1:
    if content[dot_idx-1:dot_idx] in white_list_1str: return True #content는 문자열이라서 슬라이싱형태로 사용해도 됨
  if dot_idx>=2:
    if content[dot_idx-2:dot_idx] in white_list_2str: return True
  return False
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡ

#ㅡㅡㅡㅡㅡㅡㅡ걸러낼 문장인지 판단
def is_spam(string):
  if len(string)>=10:
    if string[-10:-8]=='뉴스' and string[-4:]=='입니다.' :
      return True
  if 'MBC 뉴스는 24시간 여러분의 제보를 기다립니다' in string:
    return True
  return False
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

def extract_after_last_email(string): #문장에서 마지막으로 등장하는 이메일의 이후 부분만을 골라냄
  if find_emails(string)==[]:
    return string
  else:
    last_email=find_emails(string)[-1]
    idx=string.rfind(last_email)
    return string[idx+len(last_email):]
    
def extract_after_first_email(string): #문장에서 처음으로 등장하는 이메일의 이후 부분만을 골라냄
  if find_emails(string)==[]:
    return string
  else:
    first_email=find_emails(string)[0]
    idx=string.find(first_email)
    return string[idx+len(first_email):]

def get_end_of_news(content): #중반이후로 등장하는 첫 이메일이 기사의 종료지점을 나타낸다고 가정함.
  skip=int(len(content)/2)
  emails=find_emails(content[skip:])
  if emails==[]:
    return len(content)-1
  else:
    idx=content[skip:].find(emails[0])+skip
    return idx #이메일이 포함된 문장의, 이메일 직전 위치를 반환. 유효 문장 확인을 위해 추가 탐색 필요 (다른 함수에서 수행)

def get_title(news,title_sep='//'):
  idx=news.find(title_sep)
  return news[:idx]
def get_content(news,title_sep='//'):
  idx=news.find(title_sep)
  return news[idx+len(title_sep):]

def get_final(content, num_sentences=10): #마지막 문장들을 리스트로 반환. 끝에 줄바꿈문자 없음.
  try: #remove 등에서 오류나는, 따옴표가 제대로 닫히지 않은 기사가 아닌 경우에만 산입시킴.
    final_sentences=[] #마지막 문장들을 담음
    area_tokens=[] #마침표를 만났을 때, 그 마침표가 다른 인용구 등에 속해있는 건 아닌지 판단할 용도
    cnt=0
    # idx=len(content) #현재 읽은 글자의 인덱스 (역순으로 읽음)
    end=get_end_of_news(content)
    idx=end+1
    for char in content[end::-1]: #역순으로 탐색
      idx-=1
      if char=='.': #마침표를 읽었다면 그게 새 문장을 가리키는 것인지 판단하려함.
        if len(area_tokens)==0: #다른 인용구에 속해있지 않으면, 추가적으로 세부적으로 판단함
          if is_new_sentence(dot_idx=idx,content=content) is True: #마침표가 새 문장을 가리킨다고 판단되었을 경우
            cnt+=1
            if cnt>num_sentences: break #충분한 문장 획득했다면 종료
            else: #아니라면 새로운 빈 문장을 만들어줌
              if final_sentences==[]:
                final_sentences.insert(0,'')
              else:
                if is_spam(final_sentences[0]) is True: #잘못된 문장이었다면 무시하고 초기화함
                  final_sentences[0]=''
                  area_tokens=[]
                else: #올바른 문장이었다면 새로운 빈 문장을 삽입해줌
                  final_sentences.insert(0,'')
      if final_sentences: #하나 이상의 문장이 탐지된 적 있다면, 최근 탐지된 문장을 완성시켜감.
        if char==')':
          area_tokens.append(char)
        elif char=='(':
          area_tokens.remove(')')
        elif char=='"':
          if char not in area_tokens:
            area_tokens.append(char)
          else:
            area_tokens.remove(char)
        elif char=='”': #닫는따옴표
          area_tokens.append(char)
        elif char=='“': #여는따옴표
          area_tokens.remove('”')#닫는따옴표제거
        final_sentences[0]=''.join([char,final_sentences[0]])
    for i in range(len(final_sentences)):
      final_sentences[i]=extract_after_first_email(final_sentences[i])
      # if '주장도 나오고 있어 의료계 혼란은 당분간 이어질 것으로 보입니다.' in content:
      #   print(final_sentences[i])
      # KBS의 두 기사가 다르게 나오는 이유는 하나에서 콤마, 하나에서 마침표가 사용되었기 때문임
    return final_sentences
  except:
   return None #remove 등에서 오류나는, 따옴표가 제대로 닫히지 않은 기사이므로 고려 대상에서 제외시킴

def preprocess(file, num_sentences): #뉴스 기사 여러개를 담은 txt파일에 대한 전처리
  #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ각각의 기사를 분리
  #n번째식별토큰으로 시작하는 줄을 n번째 기사의 시작으로 간주함. n=1부터 시작하여 새 기사 확인시 증가
  #식별토큰 : 첫줄의 시작이 '1 ' 이면 '정수 '이고, 첫줄의 시작이 '1/' 이면 '정수//' 임.
  startstr=file.read(2)
  if startstr=='1 ': #기사 구별에 사용되는 구분자의 종류 식별
    news_sep_token=' '
  elif startstr=='1/':
    news_sep_token='//'
  else:
    print('오류1')
    return
  file.seek(0) #파일포인터를 파일의 시작위치로 복귀
  news_list=[] #기사들을 담을 곳
  news_index=1 #몇번째 기사인지 식별
  news_sep=str(news_index)+news_sep_token #기사 초입부분 인덱스 식별용. '1 ', '2 ', ... 또는 '1//', '2//', ... 식으로 변화
  for line in file.readlines():
    if line[0:len(news_sep)]==news_sep: #새로운 기사를 만났다면 초입부 인덱스 제외한 나머지를 news_list에 추가
      news_list.append(line[len(news_sep):-1])
      news_index+=1
      news_sep=str(news_index)+news_sep_token
    else:
      news_list[-1]=''.join([news_list[-1],line]) #이번에 읽은 줄이 기존 기사의 일부였다면 최근 획득 기사에 연장하여 덧붙여줌. 근데 보니까 각 line이 하나의 기사를 담는듯해서 이건 수행 안될지도.
  #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
  #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ분리된 기사마다 불필요한 문장 제거 후 마지막 10문장 추출
  processed_list=[] #[i][0]는 i번째 기사의 제목, [i][j]는 i번째 기사에서 추출한 문장들 중 j번째 문장
  for news in news_list:
    title=get_title(news) #문자열 반환
    content=get_content(news) #문자열 반환
    content=get_final(content, num_sentences=num_sentences) #리스트 반환
    if content is not None:
      # content=''.join(content) #문장 각각을 반환받기위해 이건 그냥 주석처리함. content를 10문장을 담은 리스트 자체로 해서 넣음.
      # processed_list[0].append(title)
      # processed_list[1].append(content)
      processed_list.append([title]+content) #[0]은 타이틀, [1] 부터는 기사의 각 문장들
    else:
      processed_list.append([title]) # 레이블 순서 안밀리게 이렇게 처리해줌
      print('내용이 없는 기사를 받았습니다')

  #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
  #ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ파일에 대한 전처리된 결과를 list로 반환
  return processed_list

#테스트셋에 사용된 뉴스 리스트 반환. "기사단위"테스트용 csv 및 "문장단위"테스트용 csv를 각각 작성. #이전버전 (txt파일 받는 버전)
# def get_test_csv(crawled_in,test_out,sample_submission_out,test_per_sentence_out,sample_submission_per_sentence_out,sep,num_sentences): 
#   if not os.path.exists(os.path.dirname(sample_submission_out)): os.makedirs(os.path.dirname(sample_submission_out))
#   if not os.path.exists(os.path.dirname(test_out)): os.makedirs(os.path.dirname(test_out))
#   if not os.path.exists(os.path.dirname(test_per_sentence_out)): os.makedirs(os.path.dirname(test_per_sentence_out))
#   if not os.path.exists(os.path.dirname(sample_submission_per_sentence_out)): os.makedirs(os.path.dirname(sample_submission_per_sentence_out))
#   total_news=[] #기사 모음
#   total_sentences=[] #문장 모음
#   #폴더 내 모든 파일에 대해 수행(재귀적하위폴더탐색X)
#   for filename in os.listdir(crawled_in):
#     if '.ipynb_checkpoints' in filename: continue
#     file1 = os.path.join(crawled_in, filename)
#     with open(file1) as file_in:
#       news_list=preprocess(file=file_in, num_sentences=num_sentences)
#     if not news_list:
#       print('오류2 (아마 줄 시작부분에 기사의 번호가 붙여져있지 않은경우. 인풋 파일을 확인하세요)')
#       continue
#     for i in range(len(news_list)): #파일마다 가지는 뉴스들을 하나로 합침. 이 상위 for는 이걸 모든 파일에 대해 수행
#       # print('title:', news_list[i][0], '///// text:', ''.join(news_list[i][1:])) #확인용
#       total_news.append(news_list[i])
#   for news in total_news:
#     for sentence in news[1:]: #제목 제외하고 text 문장만 확보하기위해 [1:] 사용
#       total_sentences.append(sentence)
    
#   index=0
#   with open(test_out, 'w') as file_out: #"기사단위" 테스트셋 생성 (문장 끝에 레이블이 없는 형식의 데이터를 갖고 생성)
#     file_out.write('index'+sep+'text\n')
#     for news in total_news:
#       file_out.write(str(index)+sep+''.join(news[1:])+'\n') #마지막 줄바꿈문자가 text에 포함안되어있어서 처리해줌. 뉴스제목은 제외시키기위해 [1:]로 슬라이싱
#       index+=1
#   index=0
#   with open(sample_submission_out, 'w') as file_out: #"기사단위" 테스트셋에 대한 답안지 생성
#     file_out.write('index'+sep+'topic_idx\n')
#     for news in total_news:
#       file_out.write(str(index)+sep+'0\n')
#       index+=1
#   index=0
#   with open(test_per_sentence_out, 'w') as file_out: #"문장단위" 테스트셋 생성 (문장 끝에 레이블이 없는 형식의 데이터를 갖고 생성)
#     file_out.write('index'+sep+'text\n')
#     for sentence in total_sentences:
#       file_out.write(str(index)+sep+sentence+'\n') 
#       index+=1
#   index=0
#   with open(sample_submission_per_sentence_out, 'w') as file_out: #"문장단위" 테스트셋에 대한 답안지 생성
#     file_out.write('index'+sep+'topic_idx\n')
#     for sentence in total_sentences:
#       file_out.write(str(index)+sep+'0\n')
#       index+=1
#   return total_news, total_sentences

# 테스트셋에 사용된 뉴스 리스트 반환. "기사단위"테스트용 csv 및 "문장단위"테스트용 csv를 각각 작성. #현재버전 (엑셀파일 받는 버전)
def get_test_csv(crawled_in,test_out,sample_submission_out,test_per_sentence_out,sample_submission_per_sentence_out,sep,num_sentences): 
  if not os.path.exists(os.path.dirname(sample_submission_out)): os.makedirs(os.path.dirname(sample_submission_out))
  if not os.path.exists(os.path.dirname(test_out)): os.makedirs(os.path.dirname(test_out))
  if not os.path.exists(os.path.dirname(test_per_sentence_out)): os.makedirs(os.path.dirname(test_per_sentence_out))
  if not os.path.exists(os.path.dirname(sample_submission_per_sentence_out)): os.makedirs(os.path.dirname(sample_submission_per_sentence_out))
  total_news=[] #기사 모음
  total_sentences=[] #문장 모음

  #희수님 엑셀파일테스트셋 => 기존 txt파일 형식으로 변환 후 10문장 추출하여 처리함
  df = pd.read_excel(crawled_in)
  df = df.iloc[:df.shape[0]] #첫줄은 인덱스로받아들이고 나머지줄은 데이터로받아들임
  labels=[]
  with open(test_out+'temp.txt','w') as file_out:
    index=1
    for text in df['문장']:
      file_out.write(str(index)+'//'+text+'\n')
      index+=1
    for label in df['후속연구']:
        labels.append(label)

  with open(test_out+'temp.txt') as file_in:
    news_list=preprocess(file=file_in, num_sentences=num_sentences)
  if not news_list:
    print('오류2 (아마 줄 시작부분에 기사의 번호가 붙여져있지 않은경우. 인풋 파일을 확인하세요)')
    return
  for i in range(len(news_list)): #파일마다 가지는 뉴스들을 하나로 합침. 이 상위 for는 이걸 모든 파일에 대해 수행
    # print('title:', news_list[i][0], '///// text:', ''.join(news_list[i][1:])) #확인용
    total_news.append(news_list[i])
  for news in total_news:
    for sentence in news[1:]: #제목 제외하고 text 문장만 확보하기위해 [1:] 사용
      total_sentences.append(sentence)
    
  index=0
  with open(test_out, 'w') as file_out: #"기사단위" 테스트셋 생성 (문장 끝에 레이블이 없는 형식의 데이터를 갖고 생성)
    file_out.write('index'+sep+'text\n')
    for news in total_news:
      file_out.write(str(index)+sep+''.join(news[1:])+'\n') #마지막 줄바꿈문자가 text에 포함안되어있어서 처리해줌. 뉴스제목은 제외시키기위해 [1:]로 슬라이싱
      index+=1
  index=0
  with open(sample_submission_out, 'w') as file_out: #"기사단위" 테스트셋에 대한 답안지 생성
    file_out.write('index'+sep+'topic_idx\n')
    for news in total_news:
      file_out.write(str(index)+sep+'0\n')
      index+=1
  index=0
  with open(test_per_sentence_out, 'w') as file_out: #"문장단위" 테스트셋 생성 (문장 끝에 레이블이 없는 형식의 데이터를 갖고 생성)
    file_out.write('index'+sep+'text\n')
    for sentence in total_sentences:
      file_out.write(str(index)+sep+sentence+'\n') 
      index+=1
  index=0
  with open(sample_submission_per_sentence_out, 'w') as file_out: #"문장단위" 테스트셋에 대한 답안지 생성
    file_out.write('index'+sep+'topic_idx\n')
    for sentence in total_sentences:
      file_out.write(str(index)+sep+'0\n')
      index+=1
  return total_news, total_sentences, labels

# 파라미터 설정 (봐야 함)

In [4]:
#입출력 관련 파라미터 설정 (굳이 안건드려도 됨)
sep=',!,!' #csv 입출력에서 column 구분용 구분자. 그냥 콤마를 구분자로하면 뉴스기사에 많이 들어있어서 안될듯
add_from_excel=True #희수님이 보내주신 수동레이블링된 엑셀을 학습용으로 추가시키려면 True, 아니면 False로 설정
num_sentences=10 #뉴스에서 추출할 문장 개수 (타이틀 제외)
num_labels=2 #레이블 가짓수. 근데 이거 바꾸면 위에 '문장의미태깅데이터셋'을 처리하는 부분도 바꿔야함. 안그러면 학습데이터의 레이블 가짓수는 그대로인데 분류할 가짓수만 변경될 것임


#입출력 관련 경로 설정 (맞게 설정해줘야 함)
project_path='/content/drive/MyDrive/딥러닝프로젝트뉴스기사/' #각종 출력 파일이 이 경로 아래에 저장됨
# crawled_in='/content/drive/MyDrive/딥러닝프로젝트뉴스기사/크롤링된데이터in/' #분류시키기 위한 테스트셋으로 사용할, 크롤링된 데이터(희수님 파일 형식)가 (다른 폴더 안거치고 직속으로) 저장된 경로 #이전버전에서 사용
crawled_in='/content/drive/MyDrive/딥러닝프로젝트뉴스기사/후속연구 데이터셋_test.xlsx'
zipfile='/content/drive/MyDrive/딥러닝프로젝트뉴스기사/문장_의미_태깅_데이터셋.zip'  #해당 압축파일 경로
excelfile='/content/drive/MyDrive/딥러닝프로젝트뉴스기사/후속연구 데이터셋.xlsx' #수동으로 레이블링한 엑셀파일(희수님 파일 형식) 경로 


#학습 관련 파라미터 설정 (조절하면서 확인해봐야함)
usage_ratio=0.1 #데이터셋을 얼마만큼 사용할건지 비율. 흐름 확인용임. 전부 학습은 오래걸리므로. 단, 수동레이블링한 엑셀파일은 모두 사용됨.
val_ratio=0.2 #학습용 데이터셋(usage_ratio가 고려된)에서 validation용으로 사용할 비율. 모델의 기본값은 0.2임. 엑셀 데이터 사용시 그것도 validation 셋에 포함될 수 있음. 이걸 변경하려면 데이터로더에 대한 이해가 필요할듯
epochs = 1 #학습 에포크. 기본값 3. 늘리면 gpu 램 부족해질지도? 확인은안해봄
batch_size = 5 #학습,발리데이션,테스트에 사용할 배치 사이즈. 기본값 128. 늘리면 gpu 램 부족해짐
max_input_length=200 #받아들일 토큰 개수. 기본값 15. 늘리면 gpu 램 부족해짐. 낮추면 텍스트의 초반부만 갖고 분류하게됨.
lr=1e-5 #러닝레이트. 기본값은 1e-5 #모델의 기본 옵티마이저는 AdamW


#기록 :
#배치 사이즈 128, 토큰최대개수 15로 할 경우 뉴스기사는 항상 첫문장에 의해서만 분류되었음
  #(기사에서 추출한 10문장 중 첫문장의 레이블이 0이면 10문장을 통째로 돌렸을 때 레이블이 0으로 분류되고, 아니면 1로 분류됨)
#데이터셋 사용비율 0.002, 배치사이즈 5일때 토큰 최대개수는 250정도 까지 가능했음 (300부터는 램 부족). 이땐 첫문장에 의해서만 분류되진 않았음.

#전처리 파일 생성 (안봐도 됨)

In [5]:
#@title
#학습용 출력 csv파일 경로 지정
topic_dict_out=os.path.join(project_path,'topic_dict.csv')
train_out=os.path.join(project_path,'train.csv') #json to csv (train)
test_out=os.path.join(project_path,'test.csv') #json to csv (test)
sample_submission_out=os.path.join(project_path,'sample_submission.csv')
test_per_sentence_out=os.path.join(project_path,'test_per_sentence.csv')
sample_submission_per_sentence_out=os.path.join(project_path,'sample_submission_per_sentence.csv')

#출력파일 생성 (학습 및 validation에 사용)
get_csv(zipfile=zipfile,excelfile=excelfile,sep=sep,add_from_excel=add_from_excel,usage_ratio=usage_ratio,train_out=train_out,topic_dict_out=topic_dict_out,num_labels=num_labels)

#출력파일 생성 (테스트셋 및 답안제출파일 sample_submission.csv)
#지금은 일단 테스트셋 레이블링이 안되어있으므로 뭐뭐로 분류되었는지 확인하기 위해 리스트로 저장해놓음
test_news,test_sentences,test_labels=get_test_csv(crawled_in=crawled_in,test_out=test_out,sample_submission_out=sample_submission_out,test_per_sentence_out=test_per_sentence_out,sample_submission_per_sentence_out=sample_submission_per_sentence_out,sep=sep,num_sentences=num_sentences) 


# for news in test_news:
#   print(''.join(news))
# for label in test_labels:
#   print(label)


#출력파일 읽어오기
train = pd.read_csv(train_out, sep=sep)
test = pd.read_csv(test_out, sep=sep)
test_per_sentence = pd.read_csv(test_per_sentence_out, sep=sep)
sample_submission = pd.read_csv(sample_submission_out, sep=sep)
sample_submission_per_sentence = pd.read_csv(sample_submission_per_sentence_out, sep=sep)
topic_dict = pd.read_csv(topic_dict_out, sep=sep)

#읽어들인 내용 확인
train.head(3)
topic_dict.head(3)

엑셀에서 추가된 데이터 개수: 59


<ipython-input-5-35e4174c7a91>:25: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train = pd.read_csv(train_out, sep=sep)
<ipython-input-5-35e4174c7a91>:26: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test = pd.read_csv(test_out, sep=sep)
<ipython-input-5-35e4174c7a91>:27: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_per_sentence = pd.read_csv(test_per_sentence_out, sep=sep)
<ipython-input-5-35e4174c7a91>

,topic,topic_idx
0,제안안함,0
1,제안함,1


# Train-Validation split (원본 ipynb에서 잘못된 부분 수정함)

In [6]:
#@title
train, val = train_test_split(train, test_size=val_ratio, random_state=2021)

In [7]:
#@title
class NTDataset(Dataset):
  
  def __init__(self, csv_file):
    self.dataset = csv_file
    self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = row[1]
    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True, # min(max_length, 로버타 라지 모델이 지원하는 최대길이) 만큼으로 자름
        max_length=max_input_length, #기본값 15. 작으면 기사에서 추출한 문장들 중 첫번째 문장에 의해서만 기사 분류가 좌우되는 것 같아서 조작해봄.
        pad_to_max_length=True, #입력 길이 안맞춰주면 문제생기는듯
        # padding='longest', #입력 길이 맞추기위해 시도해봄. 배치 내 최장길이로 피팅. padtomaxlength가 더 우선되는듯.
        # 배치 간의 길이도 일정하게 맞춰줘야 하는 듯해서 padding='longest'만으로는 안됨
        #pad_to_max_length는 곧 사라질거라함. 대신에 padding='max_length' 를 쓰라고함.
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y





class NTDataset_test(Dataset): #위에거랑 차이는 y 즉 데이터셋에 레이블이 존재하냐 아니냐에서 비롯됨
  
  def __init__(self, csv_file):
    self.dataset = csv_file
    self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:2].values
    text = row[0]
    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True, # min(max_length, 로버타 라지 모델이 지원하는 최대길이) 만큼으로 자름
        max_length=max_input_length, #기본값 15. 작으면 기사에서 추출한 문장들 중 첫번째 문장에 의해서만 기사 분류가 좌우되는 것 같아서 조작해봄.
        pad_to_max_length=True, #입력 길이 안맞춰주면 문제생기는듯
        # padding='longest', #입력 길이 맞추기위해 시도해봄. 배치 내 최장길이로 피팅. padtomaxlength가 더 우선되는듯
        # 배치 간의 길이도 일정하게 맞춰줘야 하는 듯해서 padding='longest'만으로는 안됨
        #pad_to_max_length는 곧 사라질거라함. 대신에 padding='max_length' 를 쓰라고함.
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

In [8]:
#@title
train_dataset = NTDataset(train)
val_dataset = NTDataset(val) #코드에서 기본으로 NTDataset(train) 으로 되어있었는데 잘못된 듯하여 수정했음

test_dataset = NTDataset_test(test)
test_per_sentence_dataset = NTDataset_test(test_per_sentence)

             index    topic_idx
count  6862.000000  6862.000000
mean   4284.447391     0.614690
std    2484.165494     0.486704
min       2.000000     0.000000
25%    2127.250000     0.000000
50%    4267.500000     1.000000
75%    6439.750000     1.000000
max    8577.000000     1.000000
             index    topic_idx
count  1716.000000  1716.000000
mean   4304.705711     0.617716
std    2445.748946     0.486087
min       0.000000     0.000000
25%    2208.250000     0.000000
50%    4376.500000     1.000000
75%    6392.500000     1.000000
max    8570.000000     1.000000
            index
count  104.000000
mean    51.500000
std     30.166206
min      0.000000
25%     25.750000
50%     51.500000
75%     77.250000
max    103.000000
           index
count  995.00000
mean   497.00000
std    287.37606
min      0.00000
25%    248.50000
50%    497.00000
75%    745.50000
max    994.00000


# Train_Roberta-large model

In [9]:
#@title
device = torch.device("cuda")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#레이블 개수 우리가 사용할 개수에 맞게 조정해봄. 기본값 7이었음.
model = RobertaForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=num_labels).to(device)


optimizer = AdamW(model.parameters(), lr=lr)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False) #기본값은 셔플 True인데 전처리에서 이미 셔플해주므로 여긴 False로 처리
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) #이것도 셔플 False
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
test_per_sentence_loader = DataLoader(test_per_sentence_dataset, batch_size=batch_size, shuffle=False)


# # ㅡㅡㅡㅡㅡㅡㅡㅡㅡ실험용 (패딩 처리 관련 변경한 시도)
# from torch.nn.utils.rnn import pad_sequence
# def collate_fn(batch):
#     input_ids, attention_masks = zip(*[(item[0], item[1]) for item in batch])
#     input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
#     attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)
#     if len(batch[0]) == 3:
#         y = torch.tensor([item[2] for item in batch])
#         return input_ids, attention_masks, y
#     else:
#         return input_ids, attention_masks

# optimizer = AdamW(model.parameters(), lr=1e-5)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
# test_per_sentence_loader = DataLoader(test_per_sentence_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

cuda


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
Y

# Train

In [10]:
#@title
# train
losses = []
accuracies = []
total_loss = 0.0
correct = 0
total = 0

import time
starttime=time.time()
train_times=[]
train_accs=[]
train_losses=[]
for i in range(epochs):

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss / total, "Accuracy:", correct.float() / total)



  train_accs.append(float(correct) / total) #내가추가함
  train_losses.append(total_loss / total)

  endtime=time.time() #시간측정부분 내가추가함
  train_times.append(endtime-starttime)
  starttime=endtime

  0%|          | 0/1373 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Train Loss: 0.021434788076043904 Accuracy: tensor(0.9631, device='cuda:0')


# Validation

In [11]:
#@title
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ시간절약하기 위해 주석처리해봄
# validation
model.eval()

pred = []
correct = 0
total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(val_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  pred.append(predicted)
  correct += (predicted == y_batch).sum()
  total += len(y_batch)

val_acc=float(correct)/total #내가추가함
print("val accuracy:", correct.float() / total)

  0%|          | 0/344 [00:00<?, ?it/s]

val accuracy: tensor(0.9843, device='cuda:0')


# Test

In [12]:
#@title
# test
model.eval()

pred = []

for input_ids_batch, attention_masks_batch in tqdm(test_loader):
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  pred.extend(predicted.tolist())
sample_submission.topic_idx = pred
sample_submission.to_csv(os.path.join(project_path,'roberta_large.csv'), index=False)

# test per sentence
model.eval()

pred = []

for input_ids_batch, attention_masks_batch in tqdm(test_per_sentence_loader):
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  pred.extend(predicted.tolist())
sample_submission_per_sentence.topic_idx = pred
sample_submission_per_sentence.to_csv(os.path.join(project_path,'roberta_large_per_sentence.csv'), index=False)




print(len(sample_submission.topic_idx))
print(len(test_labels))
if len(sample_submission.topic_idx)!=len(test_labels):
  print('답안지 길이와 정답지 길이가 다름')
else:
  cnt=0
  total=len(sample_submission.topic_idx)
  for i in range(len(sample_submission.topic_idx)):
    if str(sample_submission.topic_idx[i])==str(test_labels[i]):
      cnt+=1
  total_acc=cnt/total
  print('정확도:', total_acc)

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

104
104
정확도: 0.6730769230769231


# 테스트셋 분류결과 텍스트파일 생성

In [13]:
#@title
news_classified=[[] for i in range(num_labels)] #[i][j][k]는 레이블 i로 분류된 j번째 뉴스의 k번째 문장임. k=0이면 타이틀임. k=-1이면 테스트셋상에서의 인덱스임
for i in range(len(sample_submission.topic_idx)):
  appended=False
  for label in range(num_labels):
    if int(sample_submission.topic_idx[i])==label:
      test_news[i].append(i)
      news_classified[label].append(test_news[i]) #test_news[i]는 리스트임. test_news[i][0]은 i번째 기사의 타이틀, [i][n]은 추출한 문장들 중 n번째 문장임. [i][-1]은 그 기사의 인덱스임
      appended=True
      break
  if appended==False: print('문제있음')

sentence_classified=[[] for i in range(num_labels)]
for i in range(len(sample_submission_per_sentence.topic_idx)):
  appended=False
  for label in range(num_labels):
    if int(sample_submission_per_sentence.topic_idx[i])==label:
      sentence_classified[label].append(test_sentences[i]) #test_sentences[i]는 문자열임. 위의 test_news[i]는 리스트고.
      appended=True
      break
  if appended==False: print('문제있음')

index=0
with open(os.path.join(project_path,'제안안함으로분류된것.txt'),'w') as f:
  for i in range(len(news_classified[0])):
    f.write(str(index)+sep+news_classified[0][i][0]+sep+''.join(news_classified[0][i][1:-1])+'\n')
    index+=1
index=0
with open(os.path.join(project_path,'제안함으로분류된것.txt'),'w') as f:
  for i in range(len(news_classified[1])):
    f.write(str(index)+sep+news_classified[1][i][0]+sep+''.join(news_classified[1][i][1:-1])+'\n')
    index+=1

index=0
with open(os.path.join(project_path,'제안안함으로분류된것(문장).txt'), 'w') as f:
  for i in range(len(sentence_classified[0])):
    f.write(str(index)+sep+sentence_classified[0][i]+'\n')
    index+=1
index=0
with open(os.path.join(project_path,'제안함으로분류된것(문장).txt'), 'w') as f:
  for i in range(len(sentence_classified[1])):
    f.write(str(index)+sep+sentence_classified[1][i]+'\n')
    index+=1



# 테스트셋 분류결과 엑셀파일 생성

In [14]:
#@title
from openpyxl import Workbook
from openpyxl.styles import PatternFill

def make_excel(label,news_len,news_classified,sentence_classified):

  wb = Workbook()
  ws = wb.active

  yellow_fill = PatternFill(start_color="FFFFFF00", end_color="FFFFFF00", fill_type="solid")
  orange_fill = PatternFill(start_color="FFFFA500", end_color="FFFFA500", fill_type="solid")

  ws['A1'] = '정답레이블'
  ws['B1'] = '예측레이블'


  ws['C1'] = '제안한 문장 개수 (누적)'
  ws['D1'] = '제안안한 문장 개수 (누적)'
  ws['E1'] = '제안한 문장 개수'
  ws['F1'] = '제안안한 문장 개수'
  for i in range(news_len):
    if i==0:
      ws['G1'] = '기사 제목'
    else:
      ws[chr(ord('G')+i)+'1'] = '문장'+str(i)

  accum_cnt0=0
  accum_cnt1=0
  row_idx=2
  incorrect=0
  total=0

  for news in news_classified[label]:
    cnt0=0
    cnt1=0
    test_idx=int(news[-1])
    ws['A'+str(row_idx)] = test_labels[test_idx]
    ws['B'+str(row_idx)] = sample_submission.topic_idx[test_idx]
    total+=1
    if str(test_labels[test_idx])!=str(sample_submission.topic_idx[test_idx]):
      # print(test_labels[row_idx-2], sample_submission.topic_idx[row_idx-2],type(test_labels[row_idx-2]), type(sample_submission.topic_idx[row_idx-2]))
      ws['B'+str(row_idx)].fill = orange_fill
      incorrect+=1
    for i in range(len(news)-1): #news[-1]을 원래 테스트셋상에서의 인덱스를 나타내기 위해서 -1해줬음.
      if i==0:
        ws['G'+str(row_idx)] = news[0]
      else:
        cell_idx=chr(ord('G')+i)+str(row_idx)
        ws[cell_idx] = news[i]
        if news[i] in sentence_classified[1]:
          ws[cell_idx].fill = yellow_fill
          cnt1+=1
        else:
          if news[i] not in sentence_classified[0]:
            print('문제있음')
          cnt0+=1
    ws['E'+str(row_idx)] = cnt1
    ws['F'+str(row_idx)] = cnt0
    accum_cnt0+=cnt0
    accum_cnt1+=cnt1
    ws['C'+str(row_idx)] = accum_cnt1
    ws['D'+str(row_idx)] = accum_cnt0
    row_idx+=1
  
  if total!=0:
    this_acc=float(total-incorrect)/total
  else:
    this_acc='None'

  
  ws['A'+str(row_idx)] = 'train_acc'
  for i in range(len(train_accs)):
    cell_idx=chr(ord('B')+i)+str(row_idx)
    ws[cell_idx] = train_accs[i]
  row_idx+=1

  ws['A'+str(row_idx)] = 'train_loss'
  for i in range(len(train_losses)):
    cell_idx=chr(ord('B')+i)+str(row_idx)
    ws[cell_idx] = train_losses[i]
  row_idx+=1
  
  ws['A'+str(row_idx)] = 'train_time'
  for i in range(len(train_times)):
    cell_idx=chr(ord('B')+i)+str(row_idx)
    ws[cell_idx] = train_times[i]
  row_idx+=1

  ws['A'+str(row_idx)] = 'total_train_time'
  ws['B'+str(row_idx)] = sum(train_times)
  row_idx+=1


  ws['A'+str(row_idx)] = 'val_acc'
  ws['B'+str(row_idx)] = val_acc
  row_idx+=1

  ws['A'+str(row_idx)] = 'total_acc'
  ws['B'+str(row_idx)] = total_acc
  row_idx+=1
  ws['A'+str(row_idx)] = 'this_acc'
  ws['B'+str(row_idx)] = this_acc
  row_idx+=1
  ws['A'+str(row_idx)] = 'usage_ratio'
  ws['B'+str(row_idx)] = usage_ratio
  row_idx+=1
  ws['A'+str(row_idx)] = 'val_ratio'
  ws['B'+str(row_idx)] = val_ratio
  row_idx+=1
  ws['A'+str(row_idx)] = 'epochs'
  ws['B'+str(row_idx)] = epochs
  row_idx+=1
  ws['A'+str(row_idx)] = 'batch_size'
  ws['B'+str(row_idx)] = batch_size
  row_idx+=1
  ws['A'+str(row_idx)] = 'max_input_length'
  ws['B'+str(row_idx)] = max_input_length
  row_idx+=1
  ws['A'+str(row_idx)] = 'lr'
  ws['B'+str(row_idx)] = lr
  row_idx+=1



  if label==0:
    wb.save(os.path.join(project_path,'제안 안한 기사 목록 (제안 문장 하이라이트).xlsx'))
  elif label==1:
    wb.save(os.path.join(project_path,'제안한 기사 목록 (제안 문장 하이라이트).xlsx'))
  else: print('문제있음')

news_len=num_sentences+1 #타이틀포함이라 1 더해줌
make_excel(label=0,news_len=news_len,news_classified=news_classified,sentence_classified=sentence_classified)
make_excel(label=1,news_len=news_len,news_classified=news_classified,sentence_classified=sentence_classified)

# 잡다 코드 작동확인용

In [15]:
#@title
# df = pd.read_excel(excelfile)
# print(df.shape[0])
# print(df.iloc[58])
# print(dataset[testcut+818])
# print(df.columns)
# ['a','b']+['c','d']

# print(chr(ord('D')+1))

# if ' 간호법 제정안이 지난달 27일 더불어민주당을 비롯한 야권 주도로 국회 본회의를 통과한 지 20일 만이다.' in sentence_classified[0]:
#   print('a')

# print(test_per_sentence)
# print(test)

# if '문제 정의' in '"tag": "이론/모형"' or 'abc' in 'xcd':
#   print('a')

# print(str('ab'))

# print('ab'+'cdd')